In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
from datetime import datetime, date
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedShuffleSplit
from IPython.display import display
import gc
import xgboost as xgb
import operator

# Draw inline
%matplotlib inline

In [4]:
train_users_path = 'data/train_users_2.csv'
test_users_path = 'data/test_users.csv'
country_data = 'data/countries.csv'

In [5]:
# Country Data
data_contries = pd.read_csv(country_data)

print('There are {} country data\n'.format(data_contries.shape[0]))
display(data_contries)

There are 10 country data



,country_destination,lat_destination,lng_destination,distance_km,destination_km2,destination_language,language_levenshtein_distance
0,AU,-26.853388,133.275160,15297.7440,7741220.0,eng,0.00
1,CA,62.393303,-96.818146,2828.1333,9984670.0,eng,0.00
2,DE,51.165707,10.452764,7879.5680,357022.0,deu,72.61
3,ES,39.896027,-2.487694,7730.7240,505370.0,spa,92.25
4,FR,46.232193,2.209667,7682.9450,643801.0,fra,92.06
5,GB,54.633220,-3.432277,6883.6590,243610.0,eng,0.00
6,IT,41.873990,12.564167,8636.6310,301340.0,ita,89.40
7,NL,52.133057,5.295250,7524.3203,41543.0,nld,63.22
8,PT,39.553444,-7.839319,7355.2534,92090.0,por,95.45
9,US,36.966427,-95.844030,0.0000,9826675.0,eng,0.00


In [6]:
print('Maximum value of longitude is {}; while Minimum value is {}\n'.format(max(data_contries['lat_destination']),min(data_contries['lat_destination'])))
print('Maximum value of latitude is {}; while Minimum value is {}'.format(max(data_contries['lng_destination']),min(data_contries['lng_destination'])))

Maximum value of longitude is 62.393303; while Minimum value is -26.853388

Maximum value of latitude is 133.27516; while Minimum value is -96.818146


Since Latitude has the range from +90°N to -90°S, 180° totally; while Longitude has the wilder range of -180°W to 180°E, 360° totally. Here I re-define Longitude to be the value within 0° to 180° (Since -180° locates at the same point with 180°), so I replace negative value of longitude, x, to be (180+x).

In [7]:
for idx in range(data_contries.shape[0]):
    if data_contries['lng_destination'][idx] < 0:
        data_contries['lng_destination'][idx] = 360+data_contries['lng_destination'][idx]

display(data_contries)

/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,country_destination,lat_destination,lng_destination,distance_km,destination_km2,destination_language,language_levenshtein_distance
0,AU,-26.853388,133.275160,15297.7440,7741220.0,eng,0.00
1,CA,62.393303,263.181854,2828.1333,9984670.0,eng,0.00
2,DE,51.165707,10.452764,7879.5680,357022.0,deu,72.61
3,ES,39.896027,357.512306,7730.7240,505370.0,spa,92.25
4,FR,46.232193,2.209667,7682.9450,643801.0,fra,92.06
5,GB,54.633220,356.567723,6883.6590,243610.0,eng,0.00
6,IT,41.873990,12.564167,8636.6310,301340.0,ita,89.40
7,NL,52.133057,5.295250,7524.3203,41543.0,nld,63.22
8,PT,39.553444,352.160681,7355.2534,92090.0,por,95.45
9,US,36.966427,264.155970,0.0000,9826675.0,eng,0.00


In [15]:
print('Maximum value of latitude is {}; while Minimum value is {}\n'.format(max(data_contries['lat_destination']),min(data_contries['lat_destination'])))
print('Maximum value of longitude is {}; while Minimum value is {}'.format(max(data_contries['lng_destination']),min(data_contries['lng_destination'])))

Maximum value of longitude is 62.393303; while Minimum value is -26.853388

Maximum value of latitude is 177.5123055; while Minimum value is 2.209667


In [15]:
dictLat={};
dictLng={};
for idx in range(data_contries.shape[0]):
    dictLat[data_contries['country_destination'][idx]] = data_contries['lat_destination'][idx]
    dictLng[data_contries['country_destination'][idx]] = data_contries['lng_destination'][idx]

print dictLat['FR']
print dictLng

46.232193
{'FR': 2.2096669999999996, 'NL': 5.29525, 'PT': 352.160681, 'CA': 263.181854, 'DE': 10.452764, 'IT': 12.564167, 'US': 264.15597, 'AU': 133.27516, 'GB': 356.5677226, 'ES': 357.5123055}


Actual and Predict destination has Range for longitude between [0, 89.246691]
                                                                latitude   between [0, 175.3026385]
1. A perfect answer (gain = 3)
difference in longitude < 20 and
difference in latitude < 40 and

2. A good answer (gain = 2)
difference in longitude < 40 and
difference in latitude < 80 and

3. A relevant answer (gain = 1)
difference in longitude < 60 and
difference in latitude < 120 and

4. An incorrect answer (gain = 0)
Others

### Improved Metrics

In [65]:
# https://www.kaggle.com/dietcoke/score-predictions-using-ndcg
# Building on Wendy Kan's ndgc_at_k example
# https://www.kaggle.com/wendykan/airbnb-recruiting-new-user-bookings/ndcg-example
#
# you can use this script for cross-validation

import numpy as np
import pandas as pd

def dcg_at_k(r, k, method=1):
    r = np.asfarray(r)[:k]
    if r.size:
        if method == 0:
            return r[0] + np.sum(r[1:] / np.log2(np.arange(2, r.size + 1)))
        elif method == 1:
            return np.sum(r / np.log2(np.arange(2, r.size + 2)))
        else:
            raise ValueError('method must be 0 or 1.')
    return 0.


def ndcg_at_k(r, k=5, method=1):
    dcg_max = dcg_at_k(sorted(r, reverse=True), k, method)
    #print "dcg_max: " + str(dcg_max)
    #print "dcg_at_k: " + str(dcg_at_k(r, k, method))
    if not dcg_max:
        return 0.
    return dcg_at_k(r, k, method) / dcg_max


def score_predictions(preds, truth, n_modes=5):
    """
    preds: pd.DataFrame
      one row for each observation, one column for each prediction.
      Columns are sorted from left to right descending in order of likelihood.
    truth: pd.Series
      one row for each obeservation.
    """
    assert(len(preds)==len(truth))
    r = pd.DataFrame(0, index=preds.index, columns=preds.columns, dtype=np.float64)
    print truth
    for row in np.arange(preds.shape[0]):
        target=truth[row]
        for col in preds.columns:
            #print preds[col][row]
            r[col][row] = getGain(target,preds[col][row])
    
    score = pd.Series(r.apply(ndcg_at_k, axis=1, reduce=True), name='score')
    return score

In [67]:
preds = pd.DataFrame([['US','FR','AR','AA'],['FR','US','AR','AA'],['FR','FR','AR','BB']])
truth = pd.Series(['US','FR','FR'])

print score_predictions(preds, truth)

0    US
1    FR
2    FR
dtype: object
0    1.0
1    1.0
2    1.0
Name: score, dtype: float64


/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:48: FutureWarning: The reduce argument is deprecated and will be removed in a future version. You can specify result_type='reduce' to try to reduce the result to the original dimensions


In [60]:
def getGain(target, pred):
    if target in dictLat:
        tar_lat = dictLat[target]
        tar_lng = dictLng[target]
    else:
        return 0
    
    if pred in dictLat:
        pred_lat = dictLat[pred]
        pred_lng = dictLng[pred]
    else:
        return 0
    
    if abs(tar_lng-pred_lng) <= 20.0 and abs(tar_lat-pred_lat) <= 40.0:
        return 3
    elif abs(tar_lng-pred_lng) <= 40.0 and abs(tar_lat-pred_lat) <= 80.0:
        return 2
    elif abs(tar_lng-pred_lng) <= 60.0 and abs(tar_lat-pred_lat) <= 120.0:
        return 1
    else:
        return 0

#print getGain('ES','GB')

In [61]:
r = pd.DataFrame(0, index=preds.index, columns=preds.columns, dtype=np.float64)
print truth
for row in np.arange(preds.shape[0]):
    target=truth[row]
    for col in preds.columns:
        #print preds[col][row]
        r[col][row] = getGain(target,preds[col][row])

#for col in preds.columns:
#    target = truth[col]
#    print target
#    for row in np.arange(preds[col].shape[0]):
#        print row

0    US
1    US
2    FR
dtype: object


In [63]:
score = pd.Series(r.apply(ndcg_at_k, axis=1, reduce=True), name='score')

dcg_max: 3.0
dcg_at_k: 3.0
dcg_max: 3.0
dcg_at_k: 1.8927892607143724
dcg_max: 4.892789260714372
dcg_at_k: 4.892789260714372


/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:1: FutureWarning: The reduce argument is deprecated and will be removed in a future version. You can specify result_type='reduce' to try to reduce the result to the original dimensions
  """Entry point for launching an IPython kernel.


In [69]:
preds = pd.DataFrame([['US','FR']])
truth = pd.Series(['FR'])

print score_predictions(preds, truth)

0    FR
dtype: object
0    0.63093
Name: score, dtype: float64


/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:48: FutureWarning: The reduce argument is deprecated and will be removed in a future version. You can specify result_type='reduce' to try to reduce the result to the original dimensions
